## Unmasking biomacromolecular conformational dynamics from 2D analysis of subdomains vibration modes and molecular kinetics (Tutorial 2)

### From raw images

In [2]:
from PIL import Image
import numpy as np

def read_tiff(path):
    """
    path - Path to the multipage-tiff file
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return np.array(images)
#frames = read_tiff('spikes/SV 12.tif')

print(frames.shape)
X = frames.reshape(frames.shape[0],frames.shape[1]*frames.shape[1]).T

NameError: name 'frames' is not defined

In [ ]:
from pydmd import DMD, BOPDMD, EDMD
from pydmd.plotter import plot_eigs, plot_summary
%matplotlib inline

optdmd = BOPDMD(svd_rank=10)#, varpro_opts_dict={"verbose": True, "tol": 0.04}) #uncomment to add options
#dmd = DMD(svd_rank=4)
optdmd.fit(X,np.linspace(0,frames.shape[0]-1,frames.shape[0]))

# Plot a summary of the DMD results.
plot_summary(optdmd,snapshots_shape=(frames.shape[1],frames.shape[1]))

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2,3,figsize=(15,8))
axs[0,0].imshow(np.real(optdmd.reconstructed_data[:,0]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[0,1].imshow(np.real(optdmd.reconstructed_data[:,frames.shape[0]//3]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[0,2].imshow(np.real(optdmd.reconstructed_data[:,frames.shape[0]-1]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[1,0].imshow(np.real(X[:,0]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[1,1].imshow(np.real(X[:,frames.shape[0]//3]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[1,2].imshow(np.real(X[:,frames.shape[0]-1]).reshape(frames.shape[1],frames.shape[1]),cmap='inferno')
axs[0,0].set_title(f'DMD'); axs[1,0].set_title(f'Original');
axs[0,0].set_axis_off(); axs[0,1].set_axis_off();  axs[0,2].set_axis_off(); 
axs[1,0].set_axis_off(); axs[1,1].set_axis_off();  axs[1,2].set_axis_off(); 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Función para calcular el centroide de un conjunto de posiciones
def calculate_center_of_mass(positions):
    return np.mean(positions, axis=0)

# Función para calcular el radio de giro
def calculate_radius_of_gyration(positions):
    com = calculate_center_of_mass(positions)
    squared_distances = np.sum((positions - com) ** 2, axis=1)  # Distancias al cuadrado
    rg = np.sqrt(np.mean(squared_distances))  # Promedio y raíz cuadrada
    return rg

# Procesar los datos de frames para obtener posiciones (X, Y)
def get_positions_from_frames(frames):
    """
    Obtiene las posiciones (X, Y) como un array numpy para cada frame,
    asumiendo intensidades en la imagen como distribución de posiciones.
    """
    positions_per_frame = []
    for frame in frames:
        y, x = np.where(frame > 0)  # Extraer las coordenadas donde hay señal
        positions = np.vstack([x, y]).T  # Apilar X, Y como posiciones
        positions_per_frame.append(positions)
    return positions_per_frame

# Extraer posiciones de los datos originales y DMD
original_positions = get_positions_from_frames(X.T.reshape(frames.shape))
dmd_positions = get_positions_from_frames(optdmd.reconstructed_data.T.reshape(frames.shape))

# Calcular radios de giro para cada frame
rgyr_original = [calculate_radius_of_gyration(positions) for positions in original_positions]
rgyr_dmd = [calculate_radius_of_gyration(positions) for positions in dmd_positions]

# Graficar radios de giro
plt.plot(rgyr_original, label="Rg AFM ")
plt.plot(rgyr_dmd, label="Rg DMD")
plt.xlabel("Frames")
plt.ylabel("Radius of Gyration")

plt.legend()
plt.show()

In [ ]:
def calculate_rmsd(positions_ref, positions_target):
    """
    Calcula el RMSD entre dos conjuntos de posiciones.
    """
    diff = positions_ref - positions_target
    rmsd = np.sqrt(np.mean(np.sum(diff ** 2, axis=1)))
    return rmsd

def compute_rmsd_series(frames, reference_frame):
    """
    Calcula la serie temporal de RMSD respecto a un frame de referencia.
    """
    rmsd_series = []
    for i in range(len(frames)):
        positions_ref = np.column_stack(np.where(reference_frame > 0))  # Posiciones del frame de referencia
        positions_target = np.column_stack(np.where(frames[i] > 0))     # Posiciones del frame actual
        
        # Emparejar las posiciones (opcional si no coinciden en tamaño)
        min_size = min(len(positions_ref), len(positions_target))
        positions_ref, positions_target = positions_ref[:min_size], positions_target[:min_size]
        
        # Calcular RMSD
        rmsd = calculate_rmsd(positions_ref, positions_target)
        rmsd_series.append(rmsd)
    return rmsd_series

# Calcular RMSD para AFM y DMD usando el primer frame como referencia
reference_afm = frames[0]
reference_dmd = optdmd.reconstructed_data[:, 0].reshape(frames.shape[1], frames.shape[2])

rmsd_afm = compute_rmsd_series(frames, reference_afm)
rmsd_dmd = compute_rmsd_series(
    optdmd.reconstructed_data.T.reshape(frames.shape[0], frames.shape[1], frames.shape[2]),
    reference_dmd
)

# Graficar ambas series de RMSD
plt.figure(figsize=(10, 6))
plt.plot(rmsd_afm, label="RMSD AFM", color="blue")
plt.plot(rmsd_dmd, label="RMSD DMD", color="red", linestyle="--")
plt.xlabel("Frames")
plt.ylabel("RMSD")
plt.legend()
plt.show()